In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# Kaggle: https://www.kaggle.com/c/tmdb-box-office-prediction

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import collections as c

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Checking to see the expected submission format
sample1 = pd.read_csv('../input/sample_submission.csv')

print(sample1.head())

train1 = pd.read_csv('../input/train.csv')
print ("Train dataset has {} rows(samples) with {} columns(features) each.".format(*train1.shape))
print(train1.head(3))
#test = pd.read_csv('test.csv', index_col='Id')
test1 = pd.read_csv('../input/test.csv')

print ("Test dataset has {} rows(samples) with {} columns(features) each.".format(*test1.shape))
print(test1.head(3))

In [ ]:
def str_to_list(x):
    return eval(x) if x and x != '#N/A' else []

list_cols = [
    'genres',
    'belongs_to_collection',
    'production_companies',
    'production_countries',
    'spoken_languages',
    'Keywords',
    'cast',
    'crew'
]

io_params = {
    'index_col': 'id',
    'converters': {col: str_to_list for col in list_cols}
}

# We can read the train and the test in one go using the concat function
data = pd.concat(
    objs=(
        pd.read_csv('../input/train.csv', **io_params).assign(is_train=True),
        pd.read_csv('../input/test.csv', **io_params).assign(is_train=False)
    ),
    sort=False
)

In [ ]:
train1.head(1).transpose()

In [ ]:
data['collection_name']=data['belongs_to_collection'].apply(lambda x: '' if len(x)==0 else x[0]['name'] )
data['has_collection'] = data['belongs_to_collection'].apply(lambda x: 0 if len(x)==0 else 1)
data.drop(['belongs_to_collection'], axis=1, inplace=True)

In [ ]:
list_of_genres = list(data['genres'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
data['num_genres'] = data['genres'].apply(lambda x: len(x) if x != {} else 0)
data['all_genres'] = data['genres'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
top_genres = [m[0] for m in c.Counter([i for j in list_of_genres for i in j])]
for g in top_genres:
    data['genre_' + g] = data['all_genres'].apply(lambda x: 1 if g in x else 0)
data.drop(['genres'], axis=1, inplace=True)
data.drop(['all_genres'], axis=1, inplace=True)

In [ ]:
list_of_companies = list(data['production_companies'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
data['num_companies'] = data['production_companies'].apply(lambda x: len(x) if x != {} else 0)
data['all_production_companies'] = data['production_companies'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
all_companies = [m[0] for m in c.Counter([i for j in list_of_companies for i in j])]
for g in all_companies:
    data['production_company_' + g] = data['all_production_companies'].apply(lambda x: 1 if g in x else 0)
data.drop(['num_companies'], axis=1, inplace=True)
data.drop(['all_production_companies'], axis=1, inplace=True)

In [ ]:
list_of_countries = list(data['production_countries'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
                                                             
data['num_countries'] = data['production_countries'].apply(lambda x: len(x) if x != {} else 0)
data['all_countries'] = data['production_countries'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
top_countries = [m[0] for m in c.Counter([i for j in list_of_countries for i in j])]
for g in top_countries:
    data['production_country_' + g] = data['all_countries'].apply(lambda x: 1 if g in x else 0)
data.drop(['production_countries', 'all_countries'], axis=1,inplace=True)

In [ ]:
list_of_spoken_languages = list(data['spoken_languages'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
data['num_spoken_languages'] = data['spoken_languages'].apply(lambda x: len(x) if x != {} else 0)
data['all_spoken_languages'] = data['spoken_languages'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')

all_spoken_languages=[]
for m in c.Counter([i for j in list_of_spoken_languages for i in j]):
    if m=='':
        m='Unknown'
    all_spoken_languages.append(m[0])

for g in all_spoken_languages:
    data['spoken_language_' + g] = data['all_spoken_languages'].apply(lambda x: 1 if g in x else 0)
data.drop(['spoken_languages', 'all_spoken_languages'], axis=1,inplace=True)

In [ ]:
list_of_Keywords = list(data['Keywords'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
                                                             
data['num_Keywords'] = data['Keywords'].apply(lambda x: len(x) if x != {} else 0)
data['all_Keywords'] = data['Keywords'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
all_Keywords = [m[0] for m in c.Counter([i for j in list_of_Keywords for i in j])]
for g in all_Keywords:
    data['Keywords_' + g] = data['all_Keywords'].apply(lambda x: 1 if g in x else 0)
data.drop(['Keywords', 'all_Keywords'], axis=1,inplace=True)

In [ ]:
# To read an image from internet you must have the Internet Settings ON
'''from  urllib.request import urlopen
from PIL import Image
url="https://image.tmdb.org/t/p/w600_and_h900_bestv2/5VKVaTJJsyDeOzY6fLcyTo1RA9g.jpg"
image = Image.open(urlopen(url))
image'''


In [ ]:
list_of_cast_names = list(data['cast'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
list_of_cast_genders = list(data['cast'].apply(lambda x: [i['gender'] for i in x] if x != {} else []).values)
list_of_cast_characters = list(data['cast'].apply(lambda x: [i['character'] for i in x] if x != {} else []).values)

In [ ]:
data['num_cast'] = data['cast'].apply(lambda x: len(x) if x != {} else 0)
top_cast_names = [m[0] for m in c.Counter([i for j in list_of_cast_names for i in j]).most_common(15)]
for g in top_cast_names:
    data['cast_name_' + g] = data['cast'].apply(lambda x: 1 if g in str(x) else 0)


In [ ]:
data['genders_0_cast'] = data['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 0]))
data['genders_1_cast'] = data['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 1]))
data['genders_2_cast'] = data['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 2]))
top_cast_characters = [m[0] for m in c.Counter([i for j in list_of_cast_characters for i in j]).most_common(15)]
for g in top_cast_characters:
    data['cast_character_' + g] = data['cast'].apply(lambda x: 1 if g in str(x) else 0)
data.drop(['cast'], axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
list_of_crew_names = list(data['crew'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
list_of_crew_jobs = list(data['crew'].apply(lambda x: [i['job'] for i in x] if x != {} else []).values)
list_of_crew_genders = list(data['crew'].apply(lambda x: [i['gender'] for i in x] if x != {} else []).values)
list_of_crew_departments = list(data['crew'].apply(lambda x: [i['department'] for i in x] if x != {} else []).values)

data['num_crew'] = data['crew'].apply(lambda x: len(x) if x != {} else 0)
top_crew_names = [m[0] for m in c.Counter([i for j in list_of_crew_names for i in j]).most_common(15)]
for g in top_crew_names:
    data['crew_name_' + g] = data['crew'].apply(lambda x: 1 if g in str(x) else 0)
data['genders_0_crew'] = data['crew'].apply(lambda x: sum([1 for i in x if i['gender'] == 0]))
data['genders_1_crew'] = data['crew'].apply(lambda x: sum([1 for i in x if i['gender'] == 1]))
data['genders_2_crew'] = data['crew'].apply(lambda x: sum([1 for i in x if i['gender'] == 2]))
top_crew_jobs = [m[0] for m in c.Counter([i for j in list_of_crew_jobs for i in j]).most_common(15)]
for j in top_crew_jobs:
    data['jobs_' + j] = data['crew'].apply(lambda x: sum([1 for i in x if i['job'] == j]))
top_crew_departments = [m[0] for m in c.Counter([i for j in list_of_crew_departments for i in j]).most_common(15)]
for j in top_crew_departments:
    data['departments_' + j] = data['crew'].apply(lambda x: sum([1 for i in x if i['department'] == j])) 
data.drop(['crew'], axis=1, inplace=True)

In [ ]:
data['has_homepage'] = 0
data.loc[data['homepage'].isnull() == False, 'has_homepage'] = 1

In [ ]:
data.drop(columns=['homepage', 'imdb_id', 'original_language', 'original_title',
        'overview', 'poster_path', 'production_companies', 'release_date',
        'status', 'tagline', 'title', 'collection_name'],axis=1,inplace=True)

In [ ]:
#data ready
#data.head()
train=data[data['is_train']==1]
test=data[data['is_train']==0]
train.drop(['is_train'],axis=1,inplace=True)
test.drop(['is_train'],axis=1,inplace=True)

In [ ]:
train.dropna(inplace=True)
test.drop(['revenue'],axis=1,inplace=True)
testrows_with_null_value=test[test.isnull().any(axis=1)]
test.dropna(inplace=True)

In [ ]:
test.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
    
train_n=pd.DataFrame(scaler.fit_transform(train),columns=train.columns,index=train.index)
test_n=pd.DataFrame(scaler.fit_transform(test),columns=test.columns,index=test.index)
train_n.head()

In [ ]:
train_n.index

In [ ]:
eda = explore_data(train_n,None,None,None,None)
eda.explore_data_basic()
#eda.show_numerical_variable_plots()
#eda.show_categorical_variable_plots()

In [ ]:
from sklearn.model_selection import train_test_split
#id_code=train[index_variable]
y_train_d=train_n['revenue']
X_train_d=train_n.drop(['revenue'],axis=1)
#X_trast=normalized_train=normalize_data(test[numerical_variables])
X_train, X_validate, y_train, y_validate = train_test_split(X_train_d, y_train_d, test_size=0.2)

In [ ]:
from sklearn.metrics import roc_auc_score
from keras.layers import LeakyReLU,Activation
from keras.callbacks import TensorBoard
from hyperopt.mongoexp import MongoTrials
import sys

from keras import callbacks
from keras import backend as K

class MyLogger(callbacks.Callback):
    def __init__(self, n):
        self.n = 10   # print loss & acc every n epochs

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.n == 0:
            curr_loss =logs.get('loss')
            curr_mse = logs.get('mean_squared_error') * 100
            curr_mae = logs.get('mean_absolute_error') * 100
            print("epoch = %4d  loss = %0.6f  mse = %0.2f  mae = %0.2f%%" % (epoch, curr_loss, curr_mse,curr_mae))

def swish(x):
    return (K.sigmoid(x) * x)
  

my_logger = MyLogger(n=1)
space = {'choice': hp.choice('num_layers',
                    [ {'layers':'two', }
                    ]),

            'units1': hp.uniform('units1', 64,164),
            'units2': hp.uniform('units2', 64,164),

            'dropout1': hp.uniform('dropout1', .25,.75),
            'dropout2': hp.uniform('dropout2',  .25,.75),

            'batch_size' : hp.uniform('batch_size', 28,128),

            'nb_epochs' :  500,
            'optimizer': hp.choice('optimizer',['adam']),
            'activation': hp.choice('activation',['relu'])
        }

def nn_objective(params):   
    from keras.models import Sequential
    from keras.layers.core import Dense, Dropout, Activation
    from keras.optimizers import Adadelta, Adam, rmsprop

    print ('Params testing: ', params)
    model = Sequential()
   
    model.add(Dense(units=round(params['units1']),  activation=params['activation'], input_dim = X_train.shape[1], kernel_initializer = "normal")) 
   
    model.add(Dense(units=round(params['units2']),  activation=params['activation'],kernel_initializer = "normal")) 
    
    model.add(Dense(units=1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer=params['optimizer'], metrics=['mse','mae'])
     
    model.fit(X_train, y_train, epochs=round(params['nb_epochs']), validation_split=0.2, 
              batch_size=round(params['batch_size']),verbose = 0,callbacks=[my_logger])
    #print(y_validate.head())
    #y_pred = model.predict(X_validate)
    #print(y_pred[:5])
    score = model.evaluate(X_validate, y_validate, verbose=0, batch_size=15)
    print('Validation data score:', score)
    sys.stdout.flush() 
    return {'loss': score[0], 'status': STATUS_OK,'Trained_Model': model}
    

##trials = MongoTrials('mongo://localhost:1234/foo_db/jobs', exp_key='exp1')
trials = Trials()
#best = fmin(nn_objective, space, algo=tpe.suggest, max_evals=10, trials=trials)

#best_model = getBestModelfromTrials(trials)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import StratifiedKFold,cross_val_score

def base_lr_objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'n_estimators': int(params['n_estimators']),
        'learning_rate': float(params['learning_rate']),
        'max_features': int(params['max_features']),
        'min_samples_leaf': int(params['min_samples_leaf']),
        'min_samples_split': int(params['min_samples_split']),
         
    }
    
    model = GradientBoostingRegressor(**params, random_state = 0)
    model.fit(X_train, y_train)
    print(params)
    score=model.score(X_validate, y_validate)
    #score = cross_val_score(clf, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
    #print("rmse {:.3f} params {}".format(score, params))
    print(score)
    return  {'loss': score, 'status': STATUS_OK, 'Trained_Model': model}

space = {
    'max_depth': hp.quniform('max_depth', 2, 4,1),
    'n_estimators': hp.quniform('n_estimators', 20, 50,5),
    'learning_rate': hp.uniform('learning_rate', 0.05, 1.0),
    'max_features': hp.uniform('max_features', 2, 4),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 1, 5),
    'min_samples_split': hp.quniform('min_samples_split', 2,4,1),
  
}

trials = Trials()
best = fmin(fn=base_lr_objective, space=space,algo=tpe.suggest,   max_evals=10,trials=trials)
#best_model = getBestModelfromTrials(trials)
print("Hyperopt estimated optimum {}".format(best))

In [ ]:
best

In [ ]:
def getBestModelfromTrials(trials):
    valid_trial_list = [trial for trial in trials
                            if STATUS_OK == trial['result']['status']]
    losses = [ float(trial['result']['loss']) for trial in valid_trial_list]
    index_having_minumum_loss = np.argmin(losses)
    best_trial_obj = valid_trial_list[index_having_minumum_loss]
    return best_trial_obj['result']['Trained_Model']


In [ ]:
best_model = getBestModelfromTrials(trials)
best_model

In [ ]:
test_n.head()

In [ ]:
scale_target = MinMaxScaler()
y_train_d=scale_target.fit_transform(train[['revenue']])


y_pred=best_model.predict(test_n)
unscaled = scale_target.inverse_transform(pd.DataFrame(y_pred))
unscaled_df=pd.DataFrame(unscaled)
#unscaled_df.head()

subs_null = pd.DataFrame(index=testrows_with_null_value.index)
subs_null['id'] = testrows_with_null_value.index
subs_null['revenue'] = np.mean(unscaled)
#subs_null
## submission
subs = pd.DataFrame(index=test_n.index)
subs['id'] = test_n.index
subs['revenue'] = unscaled
subs_final = pd.concat([subs,subs_null], ignore_index=True)
subs_final.to_csv('submission.csv', index=False,float_format='%.0f')
subs_final.head()

In [ ]:
subs.shape

In [ ]:
'''import featuretools as ft

# Make an entityset and add the entity
es = ft.EntitySet(id = 'box_office')
es.entity_from_dataframe(entity_id = 'data', dataframe = filtered, 
                         make_index = True, index = 'index')

# Run deep feature synthesis with transformation primitives
feature_matrix, feature_defs = ft.dfs(entityset = es, target_entity = 'data',
                                      trans_primitives = ['add_numeric', 'multiply_numeric'])

feature_matrix.head()'''

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
    
def transform_data(data,normalize_variables):
    #normalize all the columns(features) so that all the values in the column lie between 0 and 1
    #this way each features will get equal preference regardless of their actual range
    n_data=pd.DataFrame(data=data)
    n_data[normalize_variables] = scaler.fit_transform(data[normalize_variables])
    #scaled = scaler.fit_transform(df)
    #unscaled = scaler.inverse_transform(scaled)
    return pd.get_dummies(n_data)

In [ ]:
rows_with_null_value=filtered[filtered.isnull().any(axis=1)]

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import StratifiedKFold,cross_val_score

def gb_objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'n_estimators': int(params['n_estimators']),
        'learning_rate': float(params['learning_rate']),
        'max_features': int(params['max_features']),
        'min_samples_leaf': int(params['min_samples_leaf']),
        'min_samples_split': int(params['min_samples_split']),
         
    }
    
    model = GradientBoostingRegressor(**params, random_state = 0)
    model.fit(X_train, y_train)
    print(params)
    score=model.score(X_validate, y_validate)
    #score = cross_val_score(clf, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
    #print("rmse {:.3f} params {}".format(score, params))
    print(score)
    return  {'loss': score, 'status': STATUS_OK, 'Trained_Model': model}

space = {
    'max_depth': hp.quniform('max_depth', 2, 4,1),
    'n_estimators': hp.quniform('n_estimators', 20, 50,5),
    'learning_rate': hp.uniform('learning_rate', 0.05, 1.0),
    'max_features': hp.uniform('max_features', 2, 4),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 1, 5),
    'min_samples_split': hp.quniform('min_samples_split', 2,4,1),
  
}

trials = Trials()
best = fmin(fn=objective, space=space,algo=tpe.suggest,   max_evals=10,trials=trials)
best_model = getBestModelfromTrials(trials)
print("Hyperopt estimated optimum {}".format(best))

In [ ]:
test.drop('revenue',inplace=True,axis=1)

In [ ]:
test_n.head()

In [ ]:
test_n.head()

In [ ]:
subs_final.head()

In [ ]:

testrows_with_null_value=rows_with_null_value[rows_with_null_value['is_train']==0]
testrows_with_null_value.shape

In [ ]:
subs_null = pd.DataFrame(index=testrows_with_null_value.index)
subs_null['id'] = testrows_with_null_value['id']
subs_null['revenue'] = np.mean(unscaled)
subs_null

In [ ]:
print(subs_final.head())
subs_final.shape

In [ ]:
#data manipulation libraries
import pandas as pd
import numpy as np
import itertools as iter

#Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import PercentFormatter
from IPython.display import display,Markdown, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from scipy.stats import chi2_contingency

class explore_data(object):
    def __init__(self, dataframe,index_variable=None,label=None,include_variables=None,exclude_variables=None):
        self.dataframe=dataframe
        if index_variable is None:
            self.index_variable=dataframe.columns[0]
        else:
            self.index_variable=index_variable
        if label is None:
            self.label=dataframe.columns[len(dataframe.columns)-1]
        else:
            self.label=index_variable
        self.include_variables=include_variables
        self.exclude_variables=exclude_variables
        self.categorical_variables=self.get_categorical_variables()
        self.numerical_variables=self.get_numerical_variables()
  
    def explore_data_basic(self):

        # a quick look at the sample of the data set
        display(('--------------------Sample Dataset--------------------'))
        display(self.dataframe.head())

        #Information on the data type
        display(('--------------------Data Types of all columns--------------------'))
        display(self.dataframe.columns.to_series().groupby(self.dataframe.dtypes).groups)

        display(('--------------------% of 0 Values per column--------------------'))
        print('{}'.format(self.dataframe.columns[self.dataframe.eq(0).any()].size))
        for col in self.dataframe.columns[self.dataframe.eq(0).any()].tolist():
        #print('{} has {} % null values'.format(col,(df[col].isnull().sum()*100.0)/df.shape[0]))
          print('{} : {} % '.format(col,(self.dataframe[col].eq(0).sum()*100.0)/self.dataframe.shape[0]))

        display(('--------------------% of Null Values per column--------------------'))
        print('{}'.format(self.dataframe.columns[self.dataframe.isnull().any()].size))
        for col in self.dataframe.columns[self.dataframe.isnull().any()].tolist():
        #print('{} has {} % null values'.format(col,(df[col].isnull().sum()*100.0)/df.shape[0]))
          print('{} : {} % '.format(col,(self.dataframe[col].isnull().sum()*100.0)/self.dataframe.shape[0]))

        display(('--------------------% of NaN/Inf Values per column--------------------'))
        #display(df.isin([np.nan, np.inf, -np.inf]).any().tolist())
        display((self.dataframe.isin([np.nan, np.inf, -np.inf]).sum()*100.0)/self.dataframe.shape[0])


        display(('--------------------Unique Values for each categorical Feature--------------------'))
        for col in list(self.dataframe.select_dtypes(include=[np.object,np.bool]).columns):
            display(self.dataframe[col].unique())

        display(('--------------------5 Point Summary of Numeric Features--------------------'))
        display(self.dataframe.describe())
    
    def show_numerical_variable_plots(self):
        self.plot_hist()
        self.box_plot()
        self.scatter_plot_independent_label()
        sns.set()
        plt.figure(figsize=(10,10)) 
        sns.heatmap(df.corr(),annot=True,cmap="Blues",fmt=".2f",linewidths=.05)
        self.pearson_correlation_matrix()
        self.check_for_outliers();
        
    def show_categorical_variable_plots(self):
        self.percent_bar_chart()
        self.chi_square_test()
        self.bar_chart()
        self.cat_plot_independent_label() 
        
    def get_categorical_variables(self): 
        cat_variables=list(self.dataframe.select_dtypes(include=[np.object,np.bool]).columns)
        if self.index_variable in cat_variables:
            cat_variables.remove(self.index_variable)
        
        if self.label is not None:
            if self.label in cat_variables :
                cat_variables.remove(label)

        if self.exclude_variables is None:
            cat_variables_filtered=cat_variables
        else:
            cat_variables_filtered=[elem for elem in cat_variables if elem not in self.exclude_variables] 
        return cat_variables_filtered;
    
    def get_numerical_variables(self): 
        numerical_variables=list(self.dataframe.select_dtypes(include=[np.object,np.bool,np.datetime64]).columns)
        if self.index_variable in numerical_variables:
            numerical_variables.remove(self.index_variable)
        
        if self.label is not None:
            if self.label in numerical_variables :
                numerical_variables.remove(label)

        if self.exclude_variables is None:
            numerical_variables_filtered=numerical_variables
        else:
            numerical_variables_filtered=[elem for elem in cat_variables if elem not in self.exclude_variables] 
        return numerical_variables_filtered;
    
        
    def plot_hist(self):
        display(Markdown('**--------------------Histograms of each Feature--------------------**'))
       
        for col in self.numerical_variables:
            plt.figure(figsize=(10,5))
            #plt.hist(dataframe[col], bins=10)
           # print(col)
            s=self.dataframe[col].dropna()
            #w = 100*(np.zeros_like(s) + 1. / len(s))
            plt.hist(s)
            plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
            plt.xlabel(col)
            plt.ylabel('Frequency')
            plt.title('Histogram of '+ col)
            plt.grid(True)
            plt.show()
        
    def bar_chart(self):
        display(('--------------------bar chart of categorial variables--------------------'))
   
        for x in self.categorical_variables:
           # if x != y:
            sns.catplot(x=x, kind="count", palette="deep", data=self.dataframe,height=5, aspect=2);
            plt.xlabel(x)
            plt.xticks(rotation='vertical')
            plt.ylabel('Frequency')
            plt.title('Bar chart of '+ x ,size=16, y=1.05)
            plt.grid(True)
            plt.show()
            display('==============================================================================================================')

    def plot_density(self):
        display(Markdown('**--------------------Density Plot of each Feature--------------------**'))
   
        for col in self.numerical_variables:
            plt.figure(figsize=(10,5))
            plt.hist(self.dataframe[col], bins=10)
            plt.xlabel(col)
            plt.ylabel('Frequency')
            sns.distplot(self.dataframe[col], hist=True, kde=True, 
                 bins=int(180/5), color = 'darkblue', 
                 hist_kws={'edgecolor':'black'},
                 kde_kws={'linewidth': 4})
            plt.grid(True)
            plt.show()
            
    def pair_plot(self):
        display(Markdown('**--------------------Pairplot of all Features with continous numeric value--------------------**'))
        for x,y in iter.combinations(include_columns,2):
            #if x != y:
                plt.figure(figsize=(10,5))
                plt.scatter(dataframe[x],dataframe[y])
                plt.xlabel(x)
                plt.ylabel(y)
                plt.title('Pairplot of '+ x + ' and ' + y)
                plt.grid(True)
                plt.show()

    def bar_chart_xyz(self):
        display(Markdown('**--------------------bar plot of each column segmented by another column--------------------**'))

        for x,y,z in iter.combinations(include_columns,3):
           # if x != y:
            sns.catplot(x=x, hue=y,col=z, kind="count", palette="deep", data=dataframe,height=20, aspect=2);
            plt.xlabel(x)
            plt.ylabel(y)
            plt.title('Bar plot of '+ x + ' and ' + y)
            plt.grid(True)
            plt.show()

    def retention_funnel(dataframe,plt,include_columns,funnel_level,aggregate_column):
        display(Markdown('**--------------------retention funnel of each column by billing cycles--------------------**'))

        for x in include_columns:
            if x !=funnel_level and x!=aggregate_column:
                df_group=dataframe.groupby([funnel_level,x])[aggregate_column].agg('count').unstack()
                pct=round((df_group.pct_change()+1)*100,0)
                #display(pct)
                pct.plot(figsize=(10,5))
                plt.xlabel(x)     

    def bar_chart_2_variables(self):
        display(('--------------------bar plot of each categorical variables segmented by another categorical variables--------------------'))

        for x,y in iter.combinations(include_columns,2):
           # if x != y:
            sns.catplot(x=x, hue=y, kind="count", palette="deep", data=dataframe,height=5, aspect=2);
            plt.xticks(rotation='vertical')
            plt.xlabel(x)
            plt.ylabel(y)
            plt.title('Bar plot of '+ x + ' and ' + y,size=16, y=1.05)
            plt.grid(True)
            plt.show()

    def bar_chart_independent_label(self):
        display(('--------------------bar plot of each categorical variables segmented by another categorical variables--------------------'))
        y=label;
        for x in include_columns:
           # if x != y:
            sns.catplot(x=x, hue=y, kind="count", palette="deep", data=dataframe,height=5, aspect=2);
            plt.xticks(rotation='vertical')
            plt.xlabel(x)
            plt.ylabel(y)
            plt.title('Bar plot of '+ x + ' and ' + y,size=16, y=1.05)
            plt.grid(True)
            plt.show()

    def bar_chart(self):
        display(Markdown('**--------------------bar plot of each column segmented by another column--------------------**'))

        for x,y in iter.combinations(include_columns,2):
           # if x != y:
            sns.catplot(x=x, hue=y, kind="count", palette="deep", data=dataframe,height=5, aspect=2);
            plt.xlabel(x)
            plt.ylabel(y)
            plt.title('Bar plot of '+ x + ' and ' + y)
            plt.grid(True)
            plt.show()

    def percent_bar_chart(self):
        display(Markdown('**--------------------bar plot of each column segmented by another column--------------------**'))

        for x,y in iter.combinations(include_columns,2):
           # if x != y:
            freq_df = dataframe.groupby([x])[y].value_counts(normalize=True).unstack()
            pct_df = freq_df.divide(freq_df.sum(axis=1), axis=0)
            pct_df.plot(kind="bar",figsize=(10,5))
            plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
            plt.legend(loc="upper right", bbox_to_anchor=(1.2,1.0))
            plt.xlabel(x)
            plt.ylabel(y)
            plt.title('Bar plot of '+ x + ' and ' + y)
            plt.grid(True)
            plt.show()

    def cat_plot_independent_label(self):
        display(('--------------------bar plot of each categorical variables segmented by another categorical variables--------------------'))
        y=label;
        for x in include_columns:
           # if x != y:
            #sns.catplot(x=x, hue=y, kind="count", palette="deep", data=dataframe,height=5, aspect=2);
            sns.catplot(x=x, y=y, data=df,height=5, aspect=2);
            plt.xticks(rotation='vertical')
            plt.xlabel(x)
            plt.ylabel(y)
            plt.title('Bar plot of '+ x + ' and ' + y,size=16, y=1.05)
            plt.grid(True)
            plt.show()


    def scatter_plot_independent_label(self):
        display(('--------------------scatter plot numerical variables by label--------------------'))
        x=label;
        for y in include_columns:
           # if x != y:
            plt.figure(figsize=(10,5))
            sns.scatterplot(x=x, y=y, palette="deep", data=dataframe);
            plt.xticks(rotation='vertical')
            plt.xlabel(x)
            plt.ylabel(y)
            plt.title('Scatter plot of '+ x + ' and ' + y,size=16, y=1.05)
            plt.grid(True)
            plt.show()

    def percent_bar_chart(self):
        display(('--------------------bar plot of each column segmented by another column--------------------'))

        for x,y in iter.combinations(include_columns,2):
           # if x != y:
            freq_df = dataframe.groupby([x])[y].value_counts(normalize=True).unstack()
            pct_df = freq_df.divide(freq_df.sum(axis=1), axis=0)
            pct_df.plot(kind="bar",figsize=(10,5))
            plt.gca().yaxis.set_major_formatter(IndexFormatter(1))
            plt.legend(loc="upper right", bbox_to_anchor=(1.2,1.0))
            plt.xlabel(x)
            plt.ylabel(y)
            plt.title('Bar plot of '+ x + ' and ' + y,size=16, y=1.05)
            plt.grid(True)
            plt.show()       

    def percentage_barh_stacked_chart(self):
        display(Markdown('**--------------------percentage chart for categorical variables--------------------**'))

        for x,y in iter.combinations(include_columns,2):
            #if x != y:
                cont_table=pd.crosstab(dataframe[x],dataframe[y], normalize='index')
                #print(cont_table)
                cont_table.plot(kind='barh', stacked=True,figsize=(10,5))
                plt.legend(loc="upper right", bbox_to_anchor=(1.2,1.0))
                plt.xlabel(x)
                plt.ylabel(y)
                plt.title('Bar plot of '+ x + ' and ' + y)
                plt.grid(True)
                plt.show()

    def line_plot_multiple(self):
        display(('--------------------time series plot of each variable --------------------'))

        for x in categorical_variables:
                plt.figure(figsize=(15,10))
                df_d=dataframe.groupby([timeseries,x], as_index=False)[value].agg({'group_size':'sum'})
                sns.lineplot(data=df_d, x=timeseries, hue=x,  y="group_size")
                plt.title('Time series plot of '+ x + ' and ' + timeseries,size=16, y=1.05)
                plt.ylabel(x)
                plt.legend(loc="upper right", bbox_to_anchor=(1.2,1.0))
                plt.xlabel(timeseries)
                plt.grid(True)
                plt.show()


    def line_plot(self):
        display(Markdown('**--------------------box plot of each column segmented by another column--------------------**'))

        for x in categorical_variables:
                plt.figure(figsize=(15,10))
                df_d=dataframe.groupby([timeseries,x], as_index=False)[value].agg({'group_size':'sum'})
                sns.lineplot(data=df_d, x=timeseries, hue=x,  y="group_size") 
                plt.legend(loc="upper right", bbox_to_anchor=(1.2,1.0))
                plt.ylabel(x)
                plt.xlabel(timeseries)
                plt.title('Time series plot of '+ x + ' and ' + timeseries)
                plt.grid(True)
                plt.show()



    def box_plot(self):
        display(Markdown('**--------------------box plot of each column segmented by another column--------------------**'))

        for x in categorical_variables:
                plt.figure(figsize=(15,10))
                sns.boxplot(x=x, y=numerical_variable,hue=segment, linewidth=2.5,data=dataframe)
                plt.legend(loc="upper right", bbox_to_anchor=(1.2,1.0))
                plt.xlabel(x)
                plt.ylabel(numerical_variable)
                plt.title('Box plot of '+ x + ' and ' + numerical_variable)
                plt.grid(True)
                plt.show()

    def box_plot_categorical_label(self):
        display(('--------------------box plot of dependent varaible for categorical variable--------------------'))

        for y in categorical_variables:
                #sns.catplot(y=label, x=y, linewidth=2.5, kind="box",data=dataframe,height=5, aspect=2)
                plt.legend(loc="upper right", bbox_to_anchor=(1.2,1.0))
                plt.xticks(rotation='vertical')
                plt.xlabel(label)
                plt.ylabel(y)
                sns.set_style("whitegrid")
                ax = sns.boxplot(x=y, y=label, data=dataframe)

                plt.title('Box plot of '+ y,size=16, y=1.05 )
                medians = dataframe.groupby([y])[label].median().values
                median_labels = [str(np.round(s, 2)) for s in medians]

                pos = range(len(medians))
                for tick,label in zip(pos,ax.get_xticklabels()):
                    ax.text(pos[tick], medians[tick] + 0.5, median_labels[tick], 
                    horizontalalignment='center', size='x-small', color='w', weight='semibold')
                plt.grid(True)
                plt.show()

    # Pearson Correlation gives the correlation and p-value

    def pearson_correlation_matrix(self):
        display(('--------------------pearson correlation--------------------'))
        corr_df = pd.DataFrame(columns=['feature', 'correlation', 'p_value'])
        rows_list = []
        y=label
        for x in include_columns:
         # if x != y:
          display('Pearson Correlation between ' + x + ' and ' + y);
          display(pearsonr(df[x],df[y]));
          display('-----------------------------------------------');
          correlation,p_value=pearsonr(df[x],df[y])
          dict1 = {}
          dict1 = {'column':x,'correlation':correlation,'p_value':p_value}
          rows_list.append(dict1)

        corr_df = pd.DataFrame(rows_list);  
        display(corr_df);

    # Correlation Matrix Heatmap
    def correlation_matrix_continuous(self):
        display(Markdown('**--------------------Correlation matrix continous numeric features--------------------**'))

        f, ax = plt.subplots(figsize=(10, 6))
        hm = sns.heatmap(round(df.corr(),2), annot=True, ax=ax, cmap="coolwarm",fmt='.2f',
                     linewidths=.05)
        f.subplots_adjust(top=0.93)
        t= f.suptitle('Correlation Heatmap', fontsize=14)   


    def chi_square_test(self):
        display(Markdown('**--------------------percentage chart for categorical variables--------------------**'))
        #final_columns=[col for col in include_columns if col not in exclude_columns]

        for x,y in iter.combinations(include_columns,2):
            #if x != y:
                crosstable=pd.crosstab(dataframe[x],dataframe[y], normalize='index')
                cont_table=pd.crosstab(dataframe[x],dataframe[y])
                #print(cont_table)
                crosstable.plot(kind='barh', stacked=True,figsize=(10,5))
                plt.legend(loc="upper right", bbox_to_anchor=(1.2,1.0))
                #plt.xlabel(x)
                plt.ylabel(x)
                plt.title('Bar plot of '+ x + ' and ' + y)
                plt.grid(True)
                plt.show()
                chi2,p,df=chi2_contingency(cont_table)[0:3]
                print('**The Null and Alternate Hypotheses**')

                print('H0:There is no statistically significant relationship between the two selected variables')
                print('Ha:There is a statistically significant relationship between the two selected variables')



                if p < .05:
                    print("We can reject the Null Hypothesis and say that " + x + " and " + y + " have some relationship")
                else:
                    print("We cannot reject the Null hypothesis and say that " + x + " and " + y + " are truly independent")
                print()
                print ("X2: {}, p-value: {}, Degrees of Freedom: {}".format(chi2,p,df))

    def check_for_outliers(self):

    # For each feature find the data points with extreme high or low values
        log_data=data[include_variables]
        total_count=len(log_data)
        x=[]
        for feature in log_data.keys():

            # TODO: Calculate Q1 (25th percentile of the data) for the given feature
            Q1 = np.percentile(log_data[feature],25)

            # TODO: Calculate Q3 (75th percentile of the data) for the given feature
            Q3 = np.percentile(log_data[feature],75)

            # TODO: Use the interquartile range to calculate an outlier step (1.5 times the interquartile range)
            step = 1.5*(Q3-Q1)
            y= log_data[~((log_data[feature] >= Q1 - step) & (log_data[feature] <= Q3 + step))]
            y1=y.index.values
            x.append(y1)

            # Display the outliers
            feature_outlier_count=y.shape[0]
            feature_percent_outliers=(float(feature_outlier_count)*100)/(float(total_count))
            print ("'{} ({:2.2f}%) Data points out of {} rows considered outliers for the feature '{}':".format(feature_outlier_count,feature_percent_outliers,total_count,feature))


            # OPTIONAL: Select the indices for data points you wish to remove
            # Here I go through the lists and extract the index value that is repeated in more than one list.
            seen = set()
            repeated = set()

        for l in x:
            for i in set(l):
                if i in seen:
                  repeated.add(i)
                else:
                  seen.add(i)

        #display dataset outlier

        if log_data.shape[1] > 1:
          outliers =list(repeated)
          outlier_count=len(outliers)
          percent_outliers=(float(outlier_count)*100)/(float(total_count))
          print("{} ({:2.2f}%) data points considered outliers from the dataset of {}.".format(outlier_count,percent_outliers,total_count))   

    def pca_results(self,n_components):

      pca = PCA(copy=True, iterated_power='auto', n_components=n_components, random_state=None,
        svd_solver='auto', tol=0.0, whiten=False)
      df_pca=df[include_variables]
      pca.fit(df_pca)


      dimensions = dimensions = ['Dimension {}'.format(i) for i in range(1,len(pca.components_)+1)]

      # PCA components
      components = pd.DataFrame(np.round(pca.components_, 4), columns = list(df_pca.keys()))
      components.index = dimensions

      # PCA explained variance
      ratios = pca.explained_variance_ratio_.reshape(len(pca.components_), 1)
      variance_ratios = pd.DataFrame(np.round(ratios, 4), columns = ['Explained Variance'])
      variance_ratios.index = dimensions

      # Create a bar plot visualization
      fig, ax = plt.subplots(figsize = (14,8))

      # Plot the feature weights as a function of the components
      components.plot(ax = ax, kind = 'bar');
      ax.set_ylabel("Feature Weights")
      ax.set_xticklabels(dimensions, rotation=0)


          # Display the explained variance ratios
      for i, ev in enumerate(pca.explained_variance_ratio_):
          ax.text(i-0.40, ax.get_ylim()[1] + 0.05, "Explained Variance\n          %.4f"%(ev))

      # Return a concatenated DataFrame
      pca_results=pd.concat([variance_ratios, components], axis = 1)

      print(pca_results['Explained Variance'].cumsum())